# **Optimization Lab**
## **Goals**
1. Hyperparameter Search using Hyperopt 


Step 1: Import Libraries

In [0]:
from numpy.random import seed
seed(2)
from tensorflow import set_random_seed
set_random_seed(2)
import tensorflow as tf
from tensorflow import keras
from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.python.data import Dataset
from sklearn import preprocessing


In [0]:
!pip install hyperopt

    100% |████████████████████████████████| 122kB 5.6MB/s 
    100% |████████████████████████████████| 409kB 7.0MB/s 


Step 2: Import Data

In [0]:
tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

#concrete_dataframe = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/slump/slump_test.data", sep=",")
concrete_dataframe = pd.read_csv("slump_test.csv", sep=",")

concrete_dataframe = concrete_dataframe.reindex(
    np.random.permutation(concrete_dataframe.index))

In [0]:
concrete_dataframe.shape[0]

103

Step 3: Preprocess

In [0]:
def preprocess_features(concrete_dataframe):
  """Prepares input features from concrete slump test data set.

  Args:
    concrete_dataframe: A Pandas DataFrame expected to contain data
      from the concrete slump test dataset.
  Returns:
    A DataFrame that contains the features to be used for the model. 
  """
  selected_features = concrete_dataframe[
    ["Cement",
     "Slag",
     "Flyash",
     "Water",
     "SP",
     "CoarseAggr.",
     "FineAggr."]]
    # Add noise
    
  processed_features = selected_features.copy()
  processed_features[processed_features.columns] = preprocessing.scale(processed_features[processed_features.columns])
  #scaler = preprocessing.StandardScaler()
  #df[df.columns] = scaler.fit_transform(df[df.columns])

  #processed_features = (selected_features - selected_features.mean()) / (selected_features.max() - selected_features.min())
    
  #processed_features["NF"] = (10000*np.random.random(concrete_dataframe.shape[0]))
  return processed_features

def preprocess_targets(concrete_dataframe):
  """Prepares target features (i.e., labels) from  housing data set.

  Args:
    dataframe: A Pandas DataFrame expected to contain data
      from the data set.
  Returns:
    A DataFrame that contains the target feature.
  """
  output_targets = concrete_dataframe["SLUMP(cm)"]
  return output_targets

Train/Test Split

In [0]:
# Choose the first 83 examples for training.
training_examples = preprocess_features(concrete_dataframe.head(83))
training_targets = preprocess_targets(concrete_dataframe.head(83))

# Choose the 20 examples for validation.
validation_examples = preprocess_features(concrete_dataframe.tail(20))
validation_targets = preprocess_targets(concrete_dataframe.tail(20))

# Double-check that we've done the right thing.
print("Training examples summary:")
display.display(training_examples.describe())
print("Validation examples summary:")
display.display(validation_examples.describe())

print("Training targets summary:")
display.display(training_targets.describe())
print("Validation targets summary:")
display.display(validation_targets.describe())



def data():
  return training_examples, training_targets, validation_examples, validation_targets

Training examples summary:


,Cement,Slag,Flyash,Water,SP,CoarseAggr.,FineAggr.
count,83.0,83.0,83.0,83.0,83.0,83.0,83.0
mean,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0
std,1.0,1.0,1.0,1.0,1.0,1.0,1.0
min,-1.2,-1.3,-1.9,-1.8,-1.4,-1.9,-1.5
25%,-1.0,-1.3,-0.4,-0.9,-0.9,-0.7,-0.9
50%,0.3,0.3,0.2,-0.1,-0.3,-0.1,0.1
75%,0.9,0.8,1.0,0.6,0.6,0.8,0.7
max,1.8,2.0,1.2,2.1,3.8,1.9,2.6


Validation examples summary:


,Cement,Slag,Flyash,Water,SP,CoarseAggr.,FineAggr.
count,20.0,20.0,20.0,20.0,20.0,20.0,20.0
mean,0.0,-0.0,0.0,0.0,0.0,0.0,-0.0
std,1.0,1.0,1.0,1.0,1.0,1.0,1.0
min,-1.0,-1.4,-1.4,-1.3,-1.2,-1.9,-1.7
25%,-0.8,-1.4,-1.4,-0.9,-1.1,-0.8,-0.7
50%,-0.6,0.3,0.2,-0.3,0.1,0.0,-0.3
75%,1.1,0.5,0.9,1.0,0.8,0.8,0.8
max,1.8,1.5,1.2,1.8,2.3,1.6,1.8


Training targets summary:


count   83.0
mean    17.5
std      9.0
min      0.0
25%     13.8
50%     21.2
75%     24.0
max     29.0
Name: SLUMP(cm), dtype: float64

Validation targets summary:


count   20.0
mean    20.1
std      7.3
min      0.0
25%     20.0
50%     22.2
75%     24.1
max     27.5
Name: SLUMP(cm), dtype: float64

Step 4: Build Model

https://www.tensorflow.org/api_docs/python/tf/keras/Model

https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense

https://keras.io/optimizers/

In [0]:
training_examples.shape[1]

#training_targets.shape

7

###Using Hyperopt for tuning Keras Models###
https://stackoverflow.com/questions/43533610/how-to-use-hyperopt-for-hyperparameter-optimization-of-keras-deep-learning-netwo

In [0]:
from __future__ import print_function

from hyperopt import Trials, STATUS_OK, tpe
from keras.datasets import mnist
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.utils import np_utils

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import roc_auc_score
import sys

Using TensorFlow backend.


In [0]:
space = {'choice': hp.choice('num_layers',
                    [ {'layers':'two', },
                    {'layers':'three',
                    'units3': 7, 
                    'dropout3': hp.uniform('dropout3', .25,.75)}
                    ]),

            'units1': 7,
            'units2': 7,

            'dropout1': hp.uniform('dropout1', .25,.75),
            'dropout2': hp.uniform('dropout2',  .25,.75),

            'batch_size' : hp.choice('batch_size',[2,8]),

            'nb_epochs' :  100,
            'optimizer': hp.choice('optimizer',['adadelta','adam','rmsprop']),
            'activation': 'relu'
        }


In [0]:
def f_nn(params): 
    print ('Params testing: ', params)
    model = Sequential()
    model.add(Dense(output_dim=params['units1'], input_dim = training_examples.shape[1])) 
    model.add(Activation(params['activation']))
    model.add(Dropout(params['dropout1']))

    model.add(Dense(output_dim=params['units2'], init = "glorot_uniform")) 
    model.add(Activation(params['activation']))
    model.add(Dropout(params['dropout2']))

    if params['choice']['layers']== 'three':
        model.add(Dense(output_dim=params['choice']['units3'], init = "glorot_uniform")) 
        model.add(Activation(params['activation']))
        model.add(Dropout(params['choice']['dropout3']))    

    model.add(Dense(1))

    model.compile(loss='mse', metrics=['mae'], optimizer=params['optimizer'])

    model.fit(training_examples, training_targets, nb_epoch=params['nb_epochs'], batch_size=params['batch_size'], verbose = 0)

    score, mae = model.evaluate(validation_examples, validation_targets, verbose=0)
    print('Test MAE:', mae)
    sys.stdout.flush() 
    return {'loss': mae, 'status': STATUS_OK}
  


In [0]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=50, trials=trials)
print ('best: ')
print (best)


Params testing:  {'activation': 'relu', 'batch_size': 8, 'choice': {'dropout3': 0.6651264044649714, 'layers': 'three', 'units3': 7}, 'dropout1': 0.5359472707759982, 'dropout2': 0.25072129569629453, 'nb_epochs': 100, 'optimizer': 'adam', 'units1': 7, 'units2': 7}


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=7, units=7)`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=7, kernel_initializer="glorot_uniform")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=7, kernel_initializer="glorot_uniform")`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Test MAE: 10.481086730957031
Params testing:  {'activation': 'relu', 'batch_size': 2, 'choice': {'dropout3': 0.5520635314188214, 'layers': 'three', 'units3': 7}, 'dropout1': 0.3309624317944963, 'dropout2': 0.4329707901335249, 'nb_epochs': 100, 'optimizer': 'adadelta', 'units1': 7, 'units2': 7}
Test MAE: 11.523927688598633
Params testing:  {'activation': 'relu', 'batch_size': 8, 'choice': {'dropout3': 0.4502539422112726, 'layers': 'three', 'units3': 7}, 'dropout1': 0.3536224543458099, 'dropout2': 0.2864336718836179, 'nb_epochs': 100, 'optimizer': 'rmsprop', 'units1': 7, 'units2': 7}
Test MAE: 9.824600219726562
Params testing:  {'activation': 'relu', 'batch_size': 8, 'choice': {'dropout3': 0.4562280537049634, 'layers': 'three', 'units3': 7}, 'dropout1': 0.31898055827055816, 'dropout2': 0.5916701716733714, 'nb_epochs': 100, 'optimizer': 'adam', 'units1': 7, 'units2': 7}
Test MAE: 10.737847328186035
Params testing:  {'activation': 'relu', 'batch_size': 2, 'choice': {'dropout3': 0.452878863

In [0]:
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 350
tf.set_random_seed(1)
#baseline_model.set_weights(binitweights)
#l1_model.set_weights(l1initweights)
#l2_model.set_weights(l2initweights)

# Store training stats
b_history = baseline_model.fit(training_examples, training_targets, epochs=EPOCHS,
                    validation_data= (validation_examples, validation_targets), verbose=0,
                    callbacks=[PrintDot()])


In [0]:
print(min(b_history.history['mean_absolute_error']))

3.1537608669464845


Step 5: Plot Results

In [0]:
import matplotlib.pyplot as plt


def plot_history(histories, key='mean_absolute_error'):
  plt.figure(figsize=(16,10))
  for name, history in histories:
    val = plt.plot(history.epoch, history.history['val_'+key],
                   '--', label=name.title()+' Val')
    plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
             label=name.title()+' Train')

  plt.xlabel('Epochs')
  plt.ylabel(key.replace('_',' ').title())
  plt.legend()

  plt.xlim([0,max(history.epoch)])
  plt.ylim([0,25])

  
plot_history([('baseline', b_history),
              ('L1', l1_history),
              ('L2', l2_history)])
  

In [0]:

nfw = l1_model.get_weights()[0][0]
y_pos = np.arange(len(nfw))
 
plt.bar(y_pos, nfw, align='center', alpha=0.5)

In [0]:
l2_model.get_weights()

##Exercise


1.   Vary L1 L2 parameters
2.   Add dropout
3.   Try to reduce val error (a) below 10 (b) below 5


